In [1]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('/content/padnas_code.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
df.head()

,Dataset,DataFrame,Columns,Task,Steps,ExpectedCode
0,Wine dataset,df,"Alcohol, Malic acid, Ash, Alcalinity of ash, M...",Identify the wine class with the lowest averag...,"Group by 'Class'., Compute the mean of 'Nonfla...",class_with_lowest_nonflavanoids = df.groupby('...
1,Breast Cancer dataset,df,"Mean radius, Mean texture, Mean perimeter, Mea...",Find the maximum 'Mean concavity' observed for...,Filter data for benign tumors (where 'Target' ...,max_concavity_benign = df[df['Target'] == 0]['...
2,Sales dataset,df,"InvoiceDocDate, CustomerCode, CustomerName, It...",Determine the item that has been sold to the m...,"Group by 'ItemCode' or 'ItemName'., Count the ...",unique_customers_per_item = df.groupby('ItemCo...
3,Adult Income dataset,df,"age, workclass, education, education-num, mari...",Determine the total number of unique occupatio...,Extract unique values from the 'occupation' co...,num_unique_occupations = df['occupation'].nuni...
4,Adult Income dataset,df,"age, workclass, education, education-num, mari...",Determine the proportion of individuals who wo...,Filter data for 'workclass' equal to 'Private'...,private_sector_high_gain_ratio = (df[df['workc...


In [4]:
INTRO_BLURB = "Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "datafrmae columns:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"

PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [5]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Dataset', 'DataFrame', 'Columns', 'Task', 'Steps', 'ExpectedCode'],
    num_rows: 60
})

In [6]:
def apply_prompt_template(examples):
  instruction = examples['Task']
  context = examples.get("Columns")
  response = examples["ExpectedCode"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
  else:
    full_prompt = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [7]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Identify the wine class with the lowest average 'Nonflavanoid phenols'.

datafrmae columns:
Alcohol, Malic acid, Ash, Alcalinity of ash, Magnesium, Total phenols, Flavanoids, Nonflavanoid phenols, Proanthocyanins, Color intensity, Hue, OD280/OD315 of diluted wines, Proline, Class

### Response:
class_with_lowest_nonflavanoids = df.groupby('Class').agg(AverageNonflavanoids=('Nonflavanoid phenols', 'mean')).idxmin().iloc[0]

### End


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [10]:
from transformers import TrainingArguments

output_dir = "results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 30
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
)


In [11]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [12]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [13]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.150800
2,2.048400
3,1.911300
4,1.547100
5,1.408500
6,1.159900
7,0.943500
8,0.799700
9,0.661800
10,0.615800


TrainOutput(global_step=30, training_loss=0.6328597192962965, metrics={'train_runtime': 161.4386, 'train_samples_per_second': 2.973, 'train_steps_per_second': 0.186, 'total_flos': 1577489497325568.0, 'train_loss': 0.6328597192962965, 'epoch': 8.0})

# Testing

In [14]:
from transformers import pipeline

In [15]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Top 5 items in term of sales money

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Top 5 items in term of sales money

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
top_sales_items = df.groupby('ItemCode').agg(TotalAmount=('TotalAmount','sum')).sort_values(by='TotalAmount', ascending=False)


None


In [16]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 most frequent customers

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 most frequent customers

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
frequent_customers = df.groupby('CustomerCode').size().top(5)


None


In [17]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 products has higest discounts

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 products has higest discounts

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
discount_per_item = df['Discount'].mean()
top_five_discounts = df.groupby('ItemCode').agg(TotalDiscount=('Discount','sum')).sort_values(by='TotalDiscount', ascending=False)


None


In [18]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
each customers spent moeny in sales

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
each customers spent moeny in sales

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
spent_per_customer = df.groupby('CustomerCode').agg(TotalAmount=('TotalAmount','sum'))


None


In [19]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
last 2 moths how much sales happend of item S101

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
last 2 moths how much sales happend of item S101

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
sales_last_2_months_item_S101 = df[(df['ItemCode'] == 'S101') & (df['DocDate'] >= dt.date(2020, 1, 1)) & (df['DocDate'] <= dt.date(2020, 3, 1))]


None


In [21]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Which item has customer F101 spent the most money on

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Which item has customer F101 spent the most money on

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
customer_total_amount_per_item = df.groupby('CustomerCode').agg(TotalAmountPerItem=('TotalAmount','sum'))
highest_total_amount_per_item_customer = customer_total_amount_per_item.idxmax().iloc[0]
highest_total_amount_per_item_customer_item = customer_total_amount_per_item.loc[highest_total_amount_per_item_customer, 'ItemCode']


None


In [24]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
how many unique items are there

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
how many unique items are there

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
unique_items = df['ItemCode'].nunique()


None


In [25]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
which item has high sales in term of money

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
which item has high sales in term of money

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
sales_by_item = df.groupby('ItemCode').agg(TotalAmount=('TotalAmount','sum')).sort_values(by='TotalAmount', ascending=False)
highest_sales_item = sales_by_item.head(1)


None


In [28]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 sales person in term of sales money

datafrmae columns:
sales person, product, customer, total amount, date

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 sales person in term of sales money

datafrmae columns:
sales person, product, customer, total amount, date

### Response:
df['SalesMoney'] = df['total amount'].apply(lambda x: x / x.iloc[1]['total amount'])
top_sales_persons = df.groupby('sales person').agg(TotalMoney=('SalesMoney','sum')).sort_values(by='TotalMoney', ascending=False)


None
